In [1]:
import pandas as pd
import base64

In [2]:
watch = [
    [
        'Elliot Page',
        'RWxsZW4=',
        'Elliot_Page',
        '2020-12-01'
    ],
    [
        'Chelsea Manning',
        'QnJhZGxleQ==',
        'Chelsea_Manning',
        '2012-08-22'
    ],
    [
        'Elke Mackenzie',
        'SXZhbg==',
        'Ivan_Mackenzie_Lamb',
        None
    ],
    [
        'Elke Mackenzie',
        'SXZhbg==',
        'Herbier_Farlow',
        None
    ],
    [
        'Jennell Jaquays',
        'UGF1bCBKYXF1YXlz',
        'Jennell_Jaquays',
        '2021-03-04'
    ],
    [
        'Wendy Carlos',
        'V2FsdGVy',
        'Wendy_Carlos',
        None
    ],
    [
        'Andréa Furet',
        'QWxleGFuZHJl',
        'Andréa_Furet',
        '2019-06-01'
    ],
]

watch

[['Elliot Page', 'RWxsZW4=', 'Elliot_Page', '2020-12-01'],
 ['Chelsea Manning', 'QnJhZGxleQ==', 'Chelsea_Manning', '2012-08-22'],
 ['Elke Mackenzie', 'SXZhbg==', 'Ivan_Mackenzie_Lamb', None],
 ['Elke Mackenzie', 'SXZhbg==', 'Herbier_Farlow', None],
 ['Jennell Jaquays', 'UGF1bCBKYXF1YXlz', 'Jennell_Jaquays', '2021-03-04'],
 ['Wendy Carlos', 'V2FsdGVy', 'Wendy_Carlos', None],
 ['Andréa Furet', 'QWxleGFuZHJl', 'Andréa_Furet', '2019-06-01']]

In [3]:
def decode(string):
    return base64.b64decode(string).decode("utf-8")

In [4]:
def load(i):
    page_name = i[2]
    
    main = (
        pd
        .read_csv(f'../data/page-revisions/{page_name.lower()}.csv')
        .set_index('revid')
        .assign(page = page_name)
        .assign(
            deadname_count = lambda df: df.added.apply(lambda x: ' '.join(eval(x)).lower().count(decode(i[1]).lower()))
        )
    )
    
    talk = (
        pd
        .read_csv(f'../data/page-revisions/discussion:{page_name.lower()}.csv')
        .set_index('revid')
        .assign(page = f'discussion:{page_name}')
        .assign(
            deadname_count = lambda df: df.added.apply(lambda x: ' '.join(eval(x)).lower().count(decode(i[1]).lower()))
        )
    )
    
    df = pd.concat([main, talk])
    return df

df = pd.concat([ load(i) for i in watch])

In [5]:
users = (
    df
    .query('timestamp >= "2024-01-15"')
    .query('deadname_count > 0')
    .groupby('user')
    .agg({
        'deadname_count': 'sum',
        'page': lambda x: ", ".join(x.unique())
    })
    .sort_values('deadname_count', ascending=False)
)

users

,deadname_count,page
user,,
Sherwood6,80,"Elliot_Page, discussion:Elliot_Page, Chelsea_M..."
GF38storic,68,"discussion:Elliot_Page, discussion:Ivan_Macken..."
Cymbella,23,"Elliot_Page, Ivan_Mackenzie_Lamb, discussion:I..."
Anne Daemon,12,"discussion:Elliot_Page, discussion:Chelsea_Man..."
Enthalpist,11,"Ivan_Mackenzie_Lamb, discussion:Ivan_Mackenzie..."
Triboulet sur une montagne,9,"Elliot_Page, Chelsea_Manning"
Lewisiscrazy,6,Andréa_Furet
JohnNewton8,4,"discussion:Elliot_Page, discussion:Chelsea_Man..."
Florck,3,discussion:Ivan_Mackenzie_Lamb


In [6]:
deadnamers = [
    'Sherwood6',
    'GF38storic',
    'Cymbella',
    'Triboulet sur une montagne'
]

In [7]:
(
    df
    .query('timestamp >= "2024-01-15"')
    .query('user.isin(@deadnamers)')
    .query('deadname_count > 0')
    [['user', 'deadname_count', 'page', 'timestamp']]
    .to_csv('../data/deadnamers.csv')
)